In [1]:
import pandas as pd
import numpy
import os
from openpyxl import load_workbook

In [12]:
df_og = pd.read_excel("Banco 2.xlsx",sheet_name='QUERY')

In [13]:
pesos = {'Cuidados ao Paciente':0.5, 'Dermocosmeticos':0.8,
       'Higiene Pessoal, Perfumaria e Cosméticos':0.5, 'Nutricosméticos':0.5,
       'OTC':0.6, 'Saúde Sexual e Saúde Masculina':0.8,
       'Tratamentos Dermatológicos':0.8, 'Cuidado Bebê/Infantil':0.8,
       'Alimentos e Bebidas':0.8, 'Conveniência':0.8, 'Prescrição':0.5}

In [16]:
df = df_og.copy()
# remove os genéricos
df = df[~df['SETOR NEC ABERTO'].str.contains("GENERICO")]

In [19]:
df

,SETOR NEC ABERTO,AREAS DA FARMACIA,LABORATORIO,MOLECULA,EAN,FCC,Produto Desc Longa,MARCA,FAMILIA,MES,Real CPP,Unidade
5108,MIP_MARCA,Cuidados ao Paciente,ABBOTT,COLAGENASE,7.896256e+12,54012,IRUXOL MONO POMADA 1.2IU 30 G x 1 (/G),IRUXOL MONO (ABT),IRUXOL (ABT),2021/02,206.67,3
5109,MIP_MARCA,Cuidados ao Paciente,ABBOTT,COLAGENASE,7.896256e+12,54012,IRUXOL MONO POMADA 1.2IU 30 G x 1 (/G),IRUXOL MONO (ABT),IRUXOL (ABT),2021/03,328.05,5
5110,MIP_MARCA,Cuidados ao Paciente,ABBOTT,COLAGENASE,7.896256e+12,54012,IRUXOL MONO POMADA 1.2IU 30 G x 1 (/G),IRUXOL MONO (ABT),IRUXOL (ABT),2021/04,68.80,1
5111,MIP_MARCA,Cuidados ao Paciente,ABBOTT,COLAGENASE,7.896256e+12,54012,IRUXOL MONO POMADA 1.2IU 30 G x 1 (/G),IRUXOL MONO (ABT),IRUXOL (ABT),2021/05,69.56,1
5112,MIP_MARCA,Cuidados ao Paciente,ABBOTT,COLAGENASE,7.896256e+12,54012,IRUXOL MONO POMADA 1.2IU 30 G x 1 (/G),IRUXOL MONO (ABT),IRUXOL (ABT),2021/06,212.49,3
...,...,...,...,...,...,...,...,...,...,...,...,...
497853,RX_TRADE,Prescrição,ZYDUS BRASIL,METIONINA| SILIMARINA,7.897460e+12,335851,SILIMALON COMPRIMIDO REVESTIDO 140MG x 60,SILIMALON (ZY-),SILIMALON (ZY-),2021/05,267.02,2
497854,RX_TRADE,Prescrição,ZYDUS BRASIL,METIONINA| SILIMARINA,7.897460e+12,335851,SILIMALON COMPRIMIDO REVESTIDO 140MG x 60,SILIMALON (ZY-),SILIMALON (ZY-),2021/06,1711.45,13
497856,RX_TRADE,Prescrição,ZYDUS BRASIL,ZOLPIDEM HEMITARTARATO,7.898928e+12,333778,ZYLINOX SL COMPRIMIDO SUBLINGUAL 5MG x 30,ZYLINOX SL (ZY-),ZYLINOX (ZY-),2021/03,83.64,2
497857,RX_TRADE,Prescrição,ZYDUS BRASIL,ZOLPIDEM HEMITARTARATO,7.898928e+12,333778,ZYLINOX SL COMPRIMIDO SUBLINGUAL 5MG x 30,ZYLINOX SL (ZY-),ZYLINOX (ZY-),2021/04,214.60,5


In [48]:
df = df_og.copy()
# remove os genéricos
df = df[~df['SETOR NEC ABERTO'].str.contains("GENERICO")]

last = df[df["MES"] >= "2020/08"]
last = last[last["MES"] < "2021/02"]
df = df[df["MES"] >= "2021/02"]

last_mean = last.groupby("FCC", as_index=False)["Unidade"].mean()
curr_mean = df.groupby("FCC", as_index=False)["Unidade"].mean()

means = curr_mean.merge(last_mean, how="left", on="FCC", suffixes=("", " Anterior")).fillna(0)
means["Deltas"] = (means["Unidade"] - means["Unidade Anterior"])
df = df.merge(means, how="left", on="FCC")
df["Deltas"] = df["Deltas"]/df["Deltas"].max()

In [49]:
df

,SETOR NEC ABERTO,AREAS DA FARMACIA,LABORATORIO,MOLECULA,EAN,FCC,Produto Desc Longa,MARCA,FAMILIA,MES,Real CPP,Unidade_x,Unidade_y,Unidade Anterior,Deltas
0,MIP_MARCA,Cuidados ao Paciente,ABBOTT,COLAGENASE,7.896256e+12,54012,IRUXOL MONO POMADA 1.2IU 30 G x 1 (/G),IRUXOL MONO (ABT),IRUXOL (ABT),2021/02,206.67,3,2.600000,2.333333,0.000333
1,MIP_MARCA,Cuidados ao Paciente,ABBOTT,COLAGENASE,7.896256e+12,54012,IRUXOL MONO POMADA 1.2IU 30 G x 1 (/G),IRUXOL MONO (ABT),IRUXOL (ABT),2021/03,328.05,5,2.600000,2.333333,0.000333
2,MIP_MARCA,Cuidados ao Paciente,ABBOTT,COLAGENASE,7.896256e+12,54012,IRUXOL MONO POMADA 1.2IU 30 G x 1 (/G),IRUXOL MONO (ABT),IRUXOL (ABT),2021/04,68.80,1,2.600000,2.333333,0.000333
3,MIP_MARCA,Cuidados ao Paciente,ABBOTT,COLAGENASE,7.896256e+12,54012,IRUXOL MONO POMADA 1.2IU 30 G x 1 (/G),IRUXOL MONO (ABT),IRUXOL (ABT),2021/05,69.56,1,2.600000,2.333333,0.000333
4,MIP_MARCA,Cuidados ao Paciente,ABBOTT,COLAGENASE,7.896256e+12,54012,IRUXOL MONO POMADA 1.2IU 30 G x 1 (/G),IRUXOL MONO (ABT),IRUXOL (ABT),2021/06,212.49,3,2.600000,2.333333,0.000333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93706,RX_TRADE,Prescrição,ZYDUS BRASIL,METIONINA| SILIMARINA,7.897460e+12,335851,SILIMALON COMPRIMIDO REVESTIDO 140MG x 60,SILIMALON (ZY-),SILIMALON (ZY-),2021/05,267.02,2,7.000000,6.000000,0.001248
93707,RX_TRADE,Prescrição,ZYDUS BRASIL,METIONINA| SILIMARINA,7.897460e+12,335851,SILIMALON COMPRIMIDO REVESTIDO 140MG x 60,SILIMALON (ZY-),SILIMALON (ZY-),2021/06,1711.45,13,7.000000,6.000000,0.001248
93708,RX_TRADE,Prescrição,ZYDUS BRASIL,ZOLPIDEM HEMITARTARATO,7.898928e+12,333778,ZYLINOX SL COMPRIMIDO SUBLINGUAL 5MG x 30,ZYLINOX SL (ZY-),ZYLINOX (ZY-),2021/03,83.64,2,4.333333,7.000000,-0.003328
93709,RX_TRADE,Prescrição,ZYDUS BRASIL,ZOLPIDEM HEMITARTARATO,7.898928e+12,333778,ZYLINOX SL COMPRIMIDO SUBLINGUAL 5MG x 30,ZYLINOX SL (ZY-),ZYLINOX (ZY-),2021/04,214.60,5,4.333333,7.000000,-0.003328


In [50]:
areas = df["AREAS DA FARMACIA"].unique()
areas

array(['Cuidados ao Paciente', 'Dermocosmeticos',
       'Higiene Pessoal, Perfumaria e Cosméticos', nan, 'Nutricosméticos',
       'OTC', 'Saúde Sexual e Saúde Masculina',
       'Tratamentos Dermatológicos', 'Cuidado Bebê/Infantil',
       'Alimentos e Bebidas', 'Conveniência', 'Prescrição'], dtype=object)

In [51]:
def incluirProduto(area, acumulado):
    peso = pesos[area]
    if acumulado <= peso:
        return 1
    else:
        return 0

In [139]:
dfCompilado = pd.DataFrame()
for area in areas:
    print(area)
    try:
        dfArea = df[df['AREAS DA FARMACIA'] == area].copy()
        areaFormatado = area.replace("/","&")[:31]
    except:
        print("PASSOU ",areaFormatado)

    dfArea = dfArea.sort_values(by="Unidade",ascending=False)
    dfArea["Part. Unidade"] = dfArea['Unidade']/dfArea['Unidade'].sum()
    dfArea['Acumulado Unidade'] = dfArea['Part. Unidade'].cumsum()

    dfArea = dfArea.sort_values(by="Real CPP",ascending=False)
    dfArea["Part. Receita"] = dfArea['Real CPP']/dfArea['Real CPP'].sum()
    dfArea['Acumulado Receita'] = dfArea['Part. Receita'].cumsum()
    
    dfArea["Incluso Unidade"] = list(map(incluirProduto,dfArea['AREAS DA FARMACIA'],dfArea["Acumulado Unidade"]))
    dfArea["Incluso Receita"] = list(map(incluirProduto,dfArea['AREAS DA FARMACIA'],dfArea["Acumulado Receita"]))
    
    dfCompilado = dfCompilado.append(dfArea)


Cuidados ao Paciente
Dermocosmeticos
Higiene Pessoal, Perfumaria e Cosméticos
Nutricosméticos
OTC
Saúde Sexual e Saúde Masculina
Tratamentos Dermatológicos
Cuidado Bebê/Infantil
Alimentos e Bebidas
Conveniência
Prescrição


In [119]:
dfCompilado['Incluso Receita'].sum()

3284

In [140]:
dfCompilado.to_excel("Compilado2.xlsx")

In [121]:
df.EAN.drop_duplicates()

609      7.896256e+12
611      7.896256e+12
613      7.896290e+12
615      7.896290e+12
617      7.896290e+12
             ...     
65067    7.898050e+12
65070    7.898050e+12
65072    7.892953e+12
65073    7.897460e+12
65074    7.898928e+12
Name: EAN, Length: 30122, dtype: float64

In [133]:
df.sort_values(by="Unidade",ascending=False)["Unidade"].describe()

count     30139.000000
mean        275.624838
std        1444.671974
min           1.000000
25%           6.000000
50%          36.000000
75%         177.000000
max      143391.000000
Name: Unidade, dtype: float64

In [134]:
df.sort_values(by="Unidade",ascending=False)["Real CPP"].describe()

count    3.013900e+04
mean     8.654983e+03
std      4.222023e+04
min      0.000000e+00
25%      1.579900e+02
50%      7.764200e+02
75%      4.340565e+03
max      3.747781e+06
Name: Real CPP, dtype: float64

In [136]:
df.sort_values(by="Real CPP",ascending=True)["Real CPP"]

12464          0.00
12620          0.00
46192          0.01
22952          0.01
35156          0.01
            ...    
57427    1120015.32
62065    1416036.52
63987    1572561.90
62067    1688147.50
62063    3747781.32
Name: Real CPP, Length: 30139, dtype: float64